# Import
---


In [0]:
import time
import pandas as pd

import torch
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets   as datasets

# Model
---
AlexNetを基にした小さい画像用CNNモデル

In [0]:
class CNN(nn.Module):
    
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(  1,  32, kernel_size=5, padding=2), nn.ReLU(inplace=True), nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d( 32,  64, kernel_size=5, padding=2), nn.ReLU(inplace=True), nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d( 64, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(), nn.Linear(256*6*6, 4096), nn.ReLU(inplace=True),
            nn.Dropout(), nn.Linear(   4096, 4096), nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(-1, 256*6*6) # flatten
        x = self.classifier(x)
        return x

In [0]:
if torch.cuda.is_available(): device = 'cuda'
else                        : device = 'cpu' 

net = CNN(num_classes=10).to(device)
net.load_state_dict(torch.load('models/MNIST.pt')) # load pretrained model by MNIST
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

# Load Dataset
---

In [4]:
toTensor  = transforms.ToTensor()
normalize = transforms.Normalize((0.5, ), (0.5, ))
transform = transforms.Compose([toTensor, normalize])

train_set = datasets.FashionMNIST(root='../data', download=True, train=True , transform=transform)
valid_set = datasets.FashionMNIST(root='../data', download=True, train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=1000, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=1000, shuffle=True)

print('size  :', train_set[0][0].size())
print('train :', len(train_set))
print('valid :', len(valid_set))

size  : torch.Size([1, 28, 28])
train : 60000
valid : 10000


# Training & Validation
---

In [0]:
def train():
    net.train()

    # mini-batch learning
    epoch_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        # forward
        t = net(x)
        # cal loss
        loss = loss_func(t, y)
        epoch_loss += loss.item()
        # backward
        loss.backward()
        # update model
        optimizer.step()
        optimizer.zero_grad()

    return epoch_loss

In [0]:
def valid():
    net.eval()

    with torch.no_grad():
        # mini-batch validation (for save memory)
        epoch_accu = 0
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            # forward
            t = net(x)
            # cal accuracy
            _, p = torch.max(t.data, dim=1)
            accu = (p == y).sum()
            epoch_accu += accu.item()

    return epoch_accu / len(valid_set)

In [7]:
max_epoch = 30

results = []
for epoch in range(max_epoch):
    st = time.time()
    loss = train()
    md = time.time()
    accu = valid()
    ed = time.time()

    results.append([epoch+1, md-st, loss, ed-md, accu])
    print(f'{epoch+1:>2} | {md - st:5.2f}s | {loss:.>6.3f} | {ed - md:5.2f}s | {accu:.2%}')

 1 | 11.35s | 58.221 |  2.02s | 81.76%
 2 | 11.77s | 24.986 |  1.58s | 86.96%
 3 | 11.64s | 19.777 |  1.58s | 87.96%
 4 | 11.36s | 17.365 |  1.56s | 88.82%


KeyboardInterrupt: ignored

# Save Model & Result
---

In [0]:
torch.save(net.state_dict(), 'models/TL_MNISTtoFMNIST.pt')

In [0]:
columns = ['epoch', 'train time(s)', 'loss', 'valid time(s)', 'accu']
df = pd.DataFrame(data=results, columns=columns).set_index('epoch')
df.to_csv('./results/TL_MNISTtoFMNIST.csv')